# Sqlalcmemy 1.4/2.0 Tutorial

## Establishing Connectity - the Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine(
    "sqlite+pysqlite:///:memory:", 
    echo=True, future=True)

## Working with Transactions and the DBAPI

### Getting a Connection

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("select 'hello world'"))
    print(result.all())

2022-04-05 19:06:29,121 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:29,123 INFO sqlalchemy.engine.Engine select 'hello world'
2022-04-05 19:06:29,126 INFO sqlalchemy.engine.Engine [generated in 0.00550s] ()
[('hello world',)]
2022-04-05 19:06:29,130 INFO sqlalchemy.engine.Engine ROLLBACK


### Committing Changes

In [3]:
with engine.connect() as conn:
    conn.execute(
        text("CREATE TABLE "
        "some_table "
        "(x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table "
        "(x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, 
        {"x": 2, "y": 4}]
    )
    conn.commit()


2022-04-05 19:06:29,290 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:29,292 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-04-05 19:06:29,293 INFO sqlalchemy.engine.Engine [generated in 0.00392s] ()
2022-04-05 19:06:29,296 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-05 19:06:29,297 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ((1, 1), (2, 4))
2022-04-05 19:06:29,299 INFO sqlalchemy.engine.Engine COMMIT


The "begin once" style

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO"
        " some_table (x, y)"
        " VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, 
        {"x": 9, "y": 10}]
    )

2022-04-05 19:06:29,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:29,498 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-05 19:06:29,500 INFO sqlalchemy.engine.Engine [cached since 0.2037s ago] ((6, 8), (9, 10))
2022-04-05 19:06:29,501 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

#### Fetching Rows

In [5]:
with engine.connect() as conn:
    result = conn.execute(
        text(
            "SELECT x, y "
            "FROM some_table"))
    for row in result:
        print(f"x: {row.x} "
        f" y: {row.y}")

2022-04-05 19:06:29,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:29,723 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-04-05 19:06:29,725 INFO sqlalchemy.engine.Engine [generated in 0.00415s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-05 19:06:29,728 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Parameters

In [6]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-05 19:06:29,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:29,961 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-04-05 19:06:29,963 INFO sqlalchemy.engine.Engine [generated in 0.00326s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-05 19:06:29,965 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Multiple Parameters

In [7]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()


2022-04-05 19:06:30,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:30,184 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-05 19:06:30,185 INFO sqlalchemy.engine.Engine [cached since 0.8888s ago] ((11, 12), (13, 14))
2022-04-05 19:06:30,187 INFO sqlalchemy.engine.Engine COMMIT


#### Bunding Parameters with a Statement

In [8]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-05 19:06:30,430 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:30,432 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-05 19:06:30,433 INFO sqlalchemy.engine.Engine [generated in 0.00326s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-05 19:06:30,436 INFO sqlalchemy.engine.Engine ROLLBACK


#### Executing with an ORM Session

In [9]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-05 19:06:30,831 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:30,833 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-05 19:06:30,834 INFO sqlalchemy.engine.Engine [cached since 0.404s ago] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-05 19:06:30,836 INFO sqlalchemy.engine.Engine ROLLBACK


commit as you go

In [10]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()


2022-04-05 19:06:31,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:31,032 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-04-05 19:06:31,033 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ((11, 9), (15, 13))
2022-04-05 19:06:31,036 INFO sqlalchemy.engine.Engine COMMIT


## Working with Database Metadata

### Setting up MetaData with Table objects

In [11]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [12]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Column

In [13]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [14]:
user_table.c.keys()

['id', 'name', 'fullname']

### Declaring Simple Constraints

In [15]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [16]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, 
    primary_key=True),
    Column('user_id',
     ForeignKey('user_account.id'), 
     nullable=False),
    Column('email_address',
     String,
     nullable=False)
)

### Emitting DDL to the Database

In [17]:
metadata_obj.create_all(engine)

2022-04-05 19:06:32,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:32,614 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-05 19:06:32,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:32,616 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-05 19:06:32,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:32,619 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-05 19:06:32,620 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:32,622 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-05 19:06:32,626 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:32,630 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-05 19:06:32,631 INFO sqlalchemy.engine.Engine [no key 0.00102s] ()
2022-04-05 19:06:32,633 INFO sqlalchemy.engine.Engine 
C

### Defining Table Metadata with the ORM

#### Setting up the Registry

In [18]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [19]:
mapper_registry.metadata

MetaData()

In [20]:
Base = mapper_registry.\
    generate_base()

#### Declaring Mapped Classes

In [21]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, 
    primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address",
     back_populates="user")
    def __repr__(self):
       return f"User(id={self.id!r}," \
       f" name={self.name!r}," \
       f" fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, 
    primary_key=True)
    email_address = Column(String,
     nullable=False)
    user_id = Column(Integer, 
    ForeignKey('user_account.id'))
    user = relationship("User", 
    back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r},"\
        f" email_address="\
        f"{self.email_address!r})"

In [22]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

#### Other Mapped Class Details

the classes have an automatically generated `__init__()` method

In [23]:
sandy = User(name='sandy',
 fullname='Sandy Cheeks')

we provided a `__repr__()` method

In [24]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

we also included a bidirectional relationship

#### Emitting DDL to the database

In [25]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)

# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

2022-04-05 19:06:34,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:34,421 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-05 19:06:34,423 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,426 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-05 19:06:34,427 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,429 INFO sqlalchemy.engine.Engine COMMIT
2022-04-05 19:06:34,430 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:34,432 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-05 19:06:34,433 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,435 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-05 19:06:34,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,440 INFO sqlalchemy.engine.Engine COMMIT


#### Combining Core Table Declarations with ORM Declarative

In [26]:
# class User(Base):
#     __table__ = user_table

#     addresses = relationship("Address", back_populates="user")

#     def __repr__(self):
#         return f"User({self.name!r}, {self.fullname!r})"

# class Address(Base):
#     __table__ = address_table

#     user = relationship("User", back_populates="addresses")

#     def __repr__(self):
#         return f"Address({self.email_address!r})"

### Table Reflection

In [27]:
some_table = Table("some_table",
 metadata_obj, autoload_with=engine)


2022-04-05 19:06:34,566 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:34,568 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-04-05 19:06:34,569 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,573 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-05 19:06:34,575 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-04-05 19:06:34,579 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-04-05 19:06:34,580 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,582 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-04-05 19:06:34,583 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-05 19:06:34,585 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [28]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

## Working With Data

### Inserting Rows with Core

The insert() SQL Expression Construct

In [29]:
from sqlalchemy import insert
stmt = insert(user_table).\
    values(name='spongebob', 
    fullname="Spongebob Squarepants")

In [30]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [31]:
compiled = stmt.compile()

In [32]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

Executing the Statement

In [33]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


2022-04-05 19:06:34,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:34,897 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-05 19:06:34,898 INFO sqlalchemy.engine.Engine [generated in 0.00270s] ('spongebob', 'Spongebob Squarepants')
2022-04-05 19:06:34,900 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
result.inserted_primary_key

(1,)

INSERT usually generates the "values" clause automatically

In [35]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", 
            "fullname": "Sandy Cheeks"},
            {"name": "patrick", 
            "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-04-05 19:06:34,994 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:34,995 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-05 19:06:34,997 INFO sqlalchemy.engine.Engine [generated in 0.00255s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-04-05 19:06:34,998 INFO sqlalchemy.engine.Engine COMMIT


Deep Alchemy

In [36]:
from sqlalchemy import select, bindparam

scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()


2022-04-05 19:06:35,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,045 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-04-05 19:06:35,046 INFO sqlalchemy.engine.Engine [generated in 0.00319s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-04-05 19:06:35,048 INFO sqlalchemy.engine.Engine COMMIT


INSERT...FROM SELECT

In [37]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)


INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


INSERT...RETURNING

In [38]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)


INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [39]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))


INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


## Selecting Rows with Core or ORM

### The select() SQL Expression Construct

In [40]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [41]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-04-05 19:06:35,279 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,281 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-05 19:06:35,282 INFO sqlalchemy.engine.Engine [generated in 0.00344s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-04-05 19:06:35,284 INFO sqlalchemy.engine.Engine ROLLBACK


In [42]:
# class User(Base):
#     __table__ = user_table

#     addresses = relationship("Address", back_populates="user")

#     def __repr__(self):
#         return f"User({self.name!r}, {self.fullname!r})"

# class Address(Base):
#     __table__ = address_table

#     user = relationship("User", back_populates="addresses")

#     def __repr__(self):
#         return f"Address({self.email_address!r})"

In [43]:
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-05 19:06:35,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,383 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-05 19:06:35,385 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-04-05 19:06:35,387 INFO sqlalchemy.engine.Engine ROLLBACK


### Setting the COLUMNS and FROM Clause

In [44]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [45]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


#### Selecting ORM Entities and Columns

In [46]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [47]:
row = session.execute(select(User)).first()

2022-04-05 19:06:35,573 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,575 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-04-05 19:06:35,577 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ()


In [48]:
row

(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

In [49]:
user = session.scalars(select(User)).first()

2022-04-05 19:06:35,664 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-04-05 19:06:35,666 INFO sqlalchemy.engine.Engine [cached since 0.09058s ago] ()


In [50]:
user

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [51]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [52]:
row = session.execute(select(User.name, User.fullname)).first()
row

2022-04-05 19:06:35,828 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-04-05 19:06:35,829 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ()


('spongebob', 'Spongebob Squarepants')

In [53]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.user_id).
    order_by(Address.id)
).all()


2022-04-05 19:06:35,878 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2022-04-05 19:06:35,878 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('sandy', Address(id=3, email_address='sandy@squirrelpower.org'))]

#### Selecting from Labeled SQL Expressions

In [54]:
from sqlalchemy import func, cast
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")


2022-04-05 19:06:35,928 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,929 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-04-05 19:06:35,931 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-04-05 19:06:35,933 INFO sqlalchemy.engine.Engine ROLLBACK


#### Select with Textual Column Expressions

In [55]:
from sqlalchemy import text
stmt = (
    select(
        text("'some phrase'"), user_table.c.name
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    print(conn.execute(stmt).all())




2022-04-05 19:06:35,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:35,981 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2022-04-05 19:06:35,982 INFO sqlalchemy.engine.Engine [generated in 0.00334s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2022-04-05 19:06:35,985 INFO sqlalchemy.engine.Engine ROLLBACK


In [56]:
from sqlalchemy import literal_column
stmt = (
    select(
        literal_column("'some phrase'").label("p"), user_table.c.name
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")




2022-04-05 19:06:36,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:36,023 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2022-04-05 19:06:36,025 INFO sqlalchemy.engine.Engine [generated in 0.00386s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2022-04-05 19:06:36,027 INFO sqlalchemy.engine.Engine ROLLBACK


### The WHERE clause

In [57]:
print(user_table.c.name == 'squidward')

user_account.name = :name_1


In [58]:
print(address_table.c.user_id > 10)

address.user_id > :user_id_1


In [59]:
print(select(user_table).where(user_table.c.name == 'squidward'))


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [60]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == 'squidward').
    where(address_table.c.user_id == user_table.c.id)
)


SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [61]:
print(
    select(address_table.c.email_address).
    where(
         user_table.c.name == 'squidward',
         address_table.c.user_id == user_table.c.id
    )
)


SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [62]:
from sqlalchemy import and_, or_
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == 'squidward', User.name == 'sandy'),
            Address.user_id == User.id
        )
    )
)


SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


In [63]:
print(
    select(User).filter_by(name='spongebob', fullname='Spongebob Squarepants')
)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


### Explicit FROM clauses and JOINs

In [64]:
print(select(user_table.c.name))


SELECT user_account.name 
FROM user_account


In [65]:
print(select(user_table.c.name, address_table.c.email_address))


SELECT user_account.name, address.email_address 
FROM user_account, address


In [66]:
with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name, address_table.c.email_address)\
            .where(user_table.c.id == address_table.c.user_id)
    )
    for row in result:
        print(row)

2022-04-05 19:06:36,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:06:36,496 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address 
FROM user_account, address 
WHERE user_account.id = address.user_id
2022-04-05 19:06:36,497 INFO sqlalchemy.engine.Engine [generated in 0.00242s] ()
('spongebob', 'spongebob@sqlalchemy.org')
('sandy', 'sandy@sqlalchemy.org')
('sandy', 'sandy@squirrelpower.org')
2022-04-05 19:06:36,499 INFO sqlalchemy.engine.Engine ROLLBACK


In [67]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [68]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)


SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [69]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)


SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [70]:
from sqlalchemy import func
print (
    select(func.count('*')).select_from(user_table)
)

SELECT count(:count_2) AS count_1 
FROM user_account


#### Setting the ON Clause

In [71]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


#### OUTER and FULL join

In [72]:
print(
    select(user_table).join(address_table, isouter=True)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [73]:
print(
    select(user_table).join(address_table, full=True)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


### ORDER BY, GROUP BY, HAVING

#### ORDER BY

In [74]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


In [75]:
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


#### Aggregate functions with GROUP BY / HAVING

In [76]:
from sqlalchemy import func
count_fn = func.count(user_table.c.id)
print(count_fn)

count(user_account.id)


In [77]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count")).
        join(Address).
        group_by(User.name).
        having(func.count(Address.id) > 1)
    )
    print(result.all())

2022-04-05 19:07:52,666 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-05 19:07:52,668 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2022-04-05 19:07:52,669 INFO sqlalchemy.engine.Engine [generated in 0.00354s] (1,)
[('sandy', 2)]
2022-04-05 19:07:52,675 INFO sqlalchemy.engine.Engine ROLLBACK
